In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import datetime as dt
import time

After imports, read in the dataset (currently only train_data)

In [4]:
# read in the datasets
train_data_path = '../csv_files/booking_train_set.csv'
# import train_data set into pandas frame
train_data = pd.read_csv(train_data_path)

In [3]:
train_data.head(10)

,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1
3,3,1006220,2016-04-16,2016-04-17,24144,desktop,384,Gondal,Gondal,1006220_1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1
5,5,1010293,2016-07-10,2016-07-11,55,mobile,359,The Devilfire Empire,Cobra Island,1010293_1
6,6,1010293,2016-07-12,2016-07-13,23921,mobile,359,The Devilfire Empire,Cobra Island,1010293_1
7,7,1010293,2016-07-13,2016-07-15,65322,desktop,9924,The Devilfire Empire,Cobra Island,1010293_1
8,8,1010293,2016-07-15,2016-07-16,23921,desktop,9924,The Devilfire Empire,Cobra Island,1010293_1
9,9,1010293,2016-07-16,2016-07-17,20545,desktop,10573,The Devilfire Empire,Cobra Island,1010293_1


Cast checkin and checkout in datetime format

In [4]:
train_data['checkin'] = pd.to_datetime(train_data['checkin'])
train_data['checkout'] = pd.to_datetime(train_data['checkout'])

Create a variable that holds the length of each 'leg' of the trip

In [5]:
train_data['lengthofleg'] = (train_data['checkout'] - train_data['checkin']).dt.days

In [6]:
train_data['lengthofleg'].head(10)

0    2
1    1
2    4
3    1
4    1
5    1
6    1
7    2
8    1
9    1
Name: lengthofleg, dtype: int64

From here, creating series that hold particular pieces of information (descriptive names)

In [7]:
# series that holds only total length of trip data for each utrip_id
lengthofstay = train_data.groupby('utrip_id')['lengthofleg'].sum()
print(lengthofstay.head(5))

utrip_id
1000027_1     8
1000033_1    10
1000045_1    11
1000083_1     5
100008_1      9
Name: lengthofleg, dtype: int64


In [8]:
# series that holds a list of cities visited on each trip
citiesvisited = train_data.groupby('utrip_id')['city_id'].agg(lambda x: list(set(x)))

In [9]:
print(citiesvisited.head(10))

utrip_id
1000027_1                          [15626, 30628, 60902, 8183]
1000033_1                         [21328, 52089, 27485, 38677]
1000045_1     [58178, 9608, 31817, 36170, 64876, 55128, 36063]
1000083_1                         [35160, 14705, 55990, 36063]
100008_1                     [12096, 6761, 11306, 65690, 6779]
1000097_1    [6306, 61187, 6788, 17127, 42503, 40521, 21033...
1000136_1                  [33540, 62541, 42482, 32627, 20345]
1000145_1                  [17764, 27112, 35850, 56651, 47499]
100018_1       [5797, 17830, 46854, 22065, 57619, 57658, 2748]
1000208_1                    [56872, 60143, 1910, 9278, 51999]
Name: city_id, dtype: object


In [10]:
# series that holds only total number of cities stopped in for each utrip_id
citiespertrip = citiesvisited.apply(lambda x: len(x))
print(citiespertrip.head(10))

utrip_id
1000027_1    4
1000033_1    4
1000045_1    7
1000083_1    4
100008_1     5
1000097_1    9
1000136_1    5
1000145_1    5
100018_1     7
1000208_1    5
Name: city_id, dtype: int64


In [11]:
# pull out highest number of cities visited
mostcitiesvisited = max(citiespertrip)
print(mostcitiesvisited)

39


In [12]:
# series that holds what countries were visited per trip
countriesvisited = train_data.groupby('utrip_id')['hotel_country'].agg(lambda x: list(set(x)))

In [13]:
print(countriesvisited.head(10))

utrip_id
1000027_1                            [Gondal]
1000033_1                      [Cobra Island]
1000045_1    [Carpathia, Fook Island, Gondal]
1000083_1                 [Gondal, Osterlich]
100008_1                           [Kamistan]
1000097_1                       [Fook Island]
1000136_1                       [Fook Island]
1000145_1              [Rook Islands, Kangan]
100018_1         [Urkesh, Bartovia, Leutonia]
1000208_1                           [Patusan]
Name: hotel_country, dtype: object


In [14]:
# series that holds total number of countries stopped in for each utrip_id
# countriespertrip = train_data.groupby('utrip_id')['hotel_country'].nunique()
countriespertrip = countriesvisited.apply(lambda x: len(x))
print(countriespertrip.head(10))

utrip_id
1000027_1    1
1000033_1    1
1000045_1    3
1000083_1    2
100008_1     1
1000097_1    1
1000136_1    1
1000145_1    2
100018_1     3
1000208_1    1
Name: hotel_country, dtype: int64


In [15]:
# pull out highest number of countries visited
mostcountriesvisited = max(countriespertrip)
print(mostcountriesvisited)

20


In [16]:
# create flag that tells us if the trip crossed any country borders
internationaltrip = countriespertrip > 1

In [17]:
# create rearranged train_data for easier visualization
train_data_rearranged = pd.DataFrame(train_data.groupby('utrip_id')['user_id'].apply(lambda x: list(set(x))[0]))
train_data_rearranged['citiesvisited'] = citiesvisited
train_data_rearranged['citiespertrip'] = citiespertrip
train_data_rearranged['countriesvisited'] = countriesvisited
train_data_rearranged['countriespertrip'] = countriespertrip
train_data_rearranged['internationaltrip'] = internationaltrip
train_data_rearranged['lengthofstay'] = lengthofstay

In [18]:
print(train_data_rearranged.head(10))

           user_id                                      citiesvisited  \
utrip_id                                                                
1000027_1  1000027                        [15626, 30628, 60902, 8183]   
1000033_1  1000033                       [21328, 52089, 27485, 38677]   
1000045_1  1000045   [58178, 9608, 31817, 36170, 64876, 55128, 36063]   
1000083_1  1000083                       [35160, 14705, 55990, 36063]   
100008_1    100008                  [12096, 6761, 11306, 65690, 6779]   
1000097_1  1000097  [6306, 61187, 6788, 17127, 42503, 40521, 21033...   
1000136_1  1000136                [33540, 62541, 42482, 32627, 20345]   
1000145_1  1000145                [17764, 27112, 35850, 56651, 47499]   
100018_1    100018    [5797, 17830, 46854, 22065, 57619, 57658, 2748]   
1000208_1  1000208                  [56872, 60143, 1910, 9278, 51999]   

           citiespertrip                  countriesvisited  countriespertrip  \
utrip_id                                   

In [19]:
# create table that holds hotel_country as key and contains all city_ids of cities inside hotel_country
citiesgroupedbycountry = train_data.groupby('hotel_country')['city_id'].agg(lambda x: list(set(x)))

In [20]:
print(citiesgroupedbycountry.head(10))

hotel_country
Absurdistan                       [33031, 52136, 42121, 40587, 62316, 60971, 170...
Aldorria                          [20610, 64131, 48136, 12169, 55059, 10134, 581...
Aldovia                           [56833, 11780, 30219, 54797, 40978, 35346, 256...
Almaigne                                                                    [41220]
Altis and Stratis, Republic of    [51329, 54918, 59398, 26376, 53767, 15242, 242...
Alvonia                           [32768, 24582, 32774, 8200, 28683, 59403, 1435...
Angrezi Raj                                                                 [64583]
Aslerfan                          [25608, 2570, 51214, 62479, 46097, 37397, 6556...
Atlantis                          [35328, 64002, 66053, 44555, 3596, 5646, 62993...
Axphain                           [24576, 53249, 10246, 20486, 65544, 26637, 206...
Name: city_id, dtype: object


In [21]:
# create table that holds hotel_country and contains all city_ids of cities inside hotel_country, ranked by number of trips there
citiesrankedbycountry = train_data.groupby('hotel_country')['city_id'].value_counts()

In [22]:
print(citiesrankedbycountry.head(50))

hotel_country  city_id
Absurdistan    52136      52
               62316       7
               5780        3
               17070       3
               47440       2
               5183        1
               27902       1
               33031       1
               40587       1
               42121       1
               51505       1
               60971       1
Aldorria       17907      49
               17638      47
               58136      35
               41035      32
               13692      26
               20862      23
               35943      19
               7969       18
               64131      18
               34751      14
               20543      13
               48136      13
               15408      12
               45760      12
               17695      11
               55059      11
               54840       8
               10134       5
               38065       5
               43084       5
               56795       4
               8060 

In [23]:
# create table that holds number of visits to hotel_country
numberofvisitstocountry = train_data['hotel_country'].value_counts()

In [24]:
print(numberofvisitstocountry.head(10))

Cobra Island              137791
Fook Island               117717
Gondal                    104979
Glubbdubdrib               74840
Elbonia                    53965
Borginia                   45629
Kangan                     44876
Bozatta                    40599
Alvonia                    34446
Holy Britannian Empire     30040
Name: hotel_country, dtype: int64


In [25]:
citiesrankedbycountry['Gondal']

city_id
36063    9559
58178    2454
37689    1371
1034     1274
30768    1271
         ... 
67344       1
67392       1
67506       1
67518       1
67551       1
Name: city_id, Length: 6440, dtype: int64

In [27]:
len(train_data['user_id'].unique())

200153

In [28]:
len(train_data['affiliate_id'].unique())

3254

In [29]:
train_data['device_class'].unique()

array(['desktop', 'mobile', 'tablet'], dtype=object)

In [32]:
train_data['booker_country'].unique()

array(['Gondal', 'The Devilfire Empire', 'Elbonia', 'Tcherkistan',
       'Bartovia'], dtype=object)

In [5]:
# series that holds a list of cities visited on each trip
orderofcitiesvisitedpertrip = train_data.groupby('utrip_id')['city_id'].agg(lambda x: list(x))

In [6]:
orderofcitiesvisitedpertrip.head(10)

utrip_id
1000027_1                          [8183, 15626, 60902, 30628]
1000033_1                  [38677, 52089, 21328, 27485, 38677]
1000045_1     [64876, 55128, 9608, 31817, 36170, 58178, 36063]
1000083_1                         [55990, 14705, 35160, 36063]
100008_1                     [11306, 12096, 6761, 6779, 65690]
1000097_1    [17127, 31088, 40521, 55128, 21033, 6306, 6788...
1000136_1                  [62541, 42482, 20345, 33540, 32627]
1000145_1                  [47499, 27112, 17764, 56651, 35850]
100018_1       [17830, 57619, 22065, 2748, 46854, 5797, 57658]
1000208_1                    [60143, 1910, 9278, 51999, 56872]
Name: city_id, dtype: object

In [65]:
len(train_data['city_id'].unique())

39901

In [66]:
lastcityoftripdf = orderofcitiesvisitedpertrip.apply(lambda x: x[-1])

In [67]:
trip_ids = orderofcitiesvisitedpertrip.index.tolist()
lastcityoftrip = lastcityoftripdf.tolist()
target_Y = list(zip(trip_ids, lastcityoftrip))
target_Y

[('1000027_1', 30628),
 ('1000033_1', 38677),
 ('1000045_1', 36063),
 ('1000083_1', 36063),
 ('100008_1', 65690),
 ('1000097_1', 42503),
 ('1000136_1', 32627),
 ('1000145_1', 35850),
 ('100018_1', 57658),
 ('1000208_1', 56872),
 ('1000212_1', 36267),
 ('1000237_1', 14288),
 ('1000241_1', 28319),
 ('1000255_1', 40502),
 ('1000309_1', 52815),
 ('1000393_2', 24752),
 ('100046_1', 52815),
 ('1000472_1', 64824),
 ('1000482_1', 4932),
 ('1000634_1', 62950),
 ('1000635_3', 1034),
 ('1000635_4', 1034),
 ('1000643_1', 62185),
 ('1000671_1', 37874),
 ('1000682_1', 1910),
 ('1000706_1', 22490),
 ('1000718_1', 66815),
 ('1000807_1', 46854),
 ('1000834_1', 48483),
 ('1000845_1', 29319),
 ('1000876_1', 17013),
 ('1000890_1', 3082),
 ('1000913_1', 38677),
 ('1000931_2', 41009),
 ('1000940_1', 10358),
 ('1000949_1', 23714),
 ('1000981_1', 4932),
 ('100099_1', 47486),
 ('1001049_1', 38023),
 ('1001059_1', 17013),
 ('1001074_2', 25025),
 ('1001092_3', 35811),
 ('1001133_2', 57658),
 ('100118_1', 45188),

In [68]:
# should not use for loop here, it's wildly inefficient
# try to use groupby and drop last row
X_train = train_data.drop(train_data.groupby('utrip_id').tail(1).index, axis=0)    

In [69]:
X_train

,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1
5,5,1010293,2016-07-10,2016-07-11,55,mobile,359,The Devilfire Empire,Cobra Island,1010293_1
...,...,...,...,...,...,...,...,...,...,...
1166828,1166828,999261,2016-09-07,2016-09-10,6306,desktop,10332,Gondal,Fook Island,999261_1
1166829,1166829,999261,2016-09-10,2016-09-13,44024,desktop,2526,Gondal,Fook Island,999261_1
1166831,1166831,999755,2016-12-13,2016-12-16,4476,desktop,2661,The Devilfire Empire,Gondal,999755_1
1166832,1166832,999755,2016-12-16,2016-12-17,1034,desktop,7974,The Devilfire Empire,Gondal,999755_1
